<a href="https://colab.research.google.com/github/shr3yans/696h/blob/main/snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 27.8 MB/s 
     |████████████████████████████████| 163 kB 61.1 MB/s 
     |████████████████████████████████| 7.6 MB 62.6 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 9.4 MB/s 
     |████████████████████████████████| 115 kB 51.5 MB/s 
     |████████████████████████████████| 212 kB 68.2 MB/s 
     |████████████████████████████████| 95 kB 5.8 MB/s 
     |████████████████████████████████| 127 kB 71.8 MB/s 
     |████████████████████████████████| 115 kB 73.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [3]:
import transformers
import torch
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

In [ ]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)

cuda:0


In [ ]:
splits = ['train[:100]', 'validation[:100]' , 'test[:100]']

raw_datasets = load_dataset("snli", split=splits)
#raw_datasets =  load_dataset("glue", "sst2")

'''
raw_datasets = raw_datasets["train"][:2]
print(raw_datasets)

train = raw_datasets["train"][:100]
validation = raw_datasets["validation"][:100]
test = raw_datasets["test"][:100]
'''

checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#print(raw_datasets)

[Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 100
}), Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 100
}), Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 100
})]


In [ ]:
# select train partition only - (idx 0 corresponds to train, 1 to validation and 2 to test)
#print("train: {} \n validation: {} \n test: {}".format(raw_datasets[0], raw_datasets[1], raw_datasets[2]))

train: Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 100
}) 
 validation: Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 100
}) 
 test: Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 100
})


In [ ]:
train_texts  = []
train_labels = []
test_texts = []
test_labels = []
for i in raw_datasets[0]:
  train_texts.append(str(i["premise"]) + '[ SEP ]' + str(i['hypothesis']))
  train_labels.append(int(i["label"]))
for i in raw_datasets[2]:
  test_texts.append(str(i["premise"]) + '[ SEP ]' + str(i['hypothesis']))
  test_labels.append(int(i["label"]))
#test_texts = raw_datasets[1]["premise"] + [' SEP '] + raw_datasets[1]['hypothesis']


In [ ]:
#len(train_texts)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True,max_length = 512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True,max_length = 512)


In [ ]:
# def tokenize_function(example):
#   return tokenizer(example["premise"], example["hypothesis"], truncation=True)
#
# tokenized_train = raw_datasets[0].map(tokenize_function, batched=True)
# tokenized_validation = raw_datasets[1].map(tokenize_function, batched=True)
# tokenized_test = raw_datasets[2].map(tokenize_function, batched=True)
# #tokenized_dataset = raw_datasets.map(tokenize_function, batched=True)

# train_encodings = tokenizer(train_texts, truncation=True, padding=True,max_length = 512)
#val_encodings = tokenizer(val_texts, truncation=True, padding=True)
# test_encodings = tokenizer(test_texts, truncation=True, padding=True,max_length = 512)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
train_set = Dataset(train_encodings, train_labels)
test_set = Dataset(test_encodings, test_labels)


# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#train_set

In [ ]:
#tokenized_train[0] # print first example in tokenized_train

In [ ]:
'''
training_args = TrainingArguments("test-trainer")

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
'''

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classi

In [ ]:
#trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=39, training_loss=1.1653559758112981, metrics={'train_runtime': 644.3492, 'train_samples_per_second': 0.466, 'train_steps_per_second': 0.061, 'total_flos': 29071988335368.0, 'train_loss': 1.1653559758112981, 'epoch': 3.0})

In [ ]:
#predictions = trainer.predict(tokenized_validation)
#print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 8


IndexError: ignored

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("snli")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model.to(device)
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-0

RuntimeError: ignored

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: premise, hypothesis. If premise, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode

Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: premise, hypothesis. If premise, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


RuntimeError: ignored